In [18]:
import numpy as np
from mmengine.structures import InstanceData
from mmpose.structures import PoseDataSample
from mmpose.visualization import PoseLocalVisualizer
from PIL import Image

In [19]:
from mmengine.dataset import BaseDataset
import cv2

class dataset300v(BaseDataset):
    # 感觉就是将标签文件中的data_list进行处理后再送给某个对象（但我这里没做任何处理）
    def parse_data_info(self, raw_data_info):
        return raw_data_info

class LoadImage:
    def __call__(self, results):
        results['img'] = cv2.imread(results['img_path'])
        return results

class ParseImage:
    def __call__(self, results):
        results['img_shape'] = results['img'].shape
        return results

pipeline = [
    LoadImage(),
    ParseImage(),
]
dataset = dataset300v(
    ann_file='E:\\mmpose\\data\\300vw\\annotations\\train.json',
    pipeline=pipeline
)
print(dataset.get_data_info(0))

{'img_path': 'E:\\mmpose\\data\\300vw\\images\\001\\000040.jpg', 'keypoints': [758.892, 197.093, 759.165, 214.967, 761.726, 232.459, 766.015, 249.821, 772.551, 265.568, 782.446, 279.961, 794.453, 292.07, 807.801, 302.202, 822.623, 306.365, 838.665, 304.6, 854.471, 295.724, 868.569, 283.871, 879.999, 269.291, 887.411, 252.227, 891.952, 234.07, 895.035, 215.161, 896.093, 195.974, 768.14, 176.124, 776.279, 170.349, 786.665, 169.484, 797.311, 172.217, 807.043, 177.421, 831.563, 178.087, 843.056, 173.857, 855.159, 172.125, 866.972, 174.701, 876.198, 181.401, 819.355, 196.024, 819.142, 208.149, 818.749, 220.104, 818.496, 232.254, 807.975, 241.414, 813.359, 243.661, 819.153, 245.223, 825.147, 244.12, 831.01, 242.79, 780.518, 195.465, 787.46, 191.453, 796.565, 192.083, 804.264, 198.614, 795.839, 200.596, 786.739, 200.232, 837.573, 200.178, 845.142, 194.213, 854.575, 194.579, 862.343, 199.109, 855.189, 203.12, 845.819, 202.973, 799.549, 266.586, 806.272, 260.192, 814.042, 256.692, 819.402, 258.

In [20]:
# 取出标注文件中的关键点
sample1 = dataset.get_data_info(20)
sample1_keypoints = sample1['keypoints']
sample1_keypoints = [[sample1_keypoints[i],sample1_keypoints[i + 1]] for i in range(0, 68*2, 2)]
print(sample1_keypoints)

# 取出标注文件中的bbox
sample1_bbox = sample1['bbox']
# 将xywh->xyxy
sample1_bbox[2] = sample1_bbox[0] + sample1_bbox[2]
sample1_bbox[3] = sample1_bbox[1] - sample1_bbox[3]
print(sample1_bbox)

[[727.809, 190.622], [726.851, 207.834], [727.886, 225.185], [730.824, 242.31], [736.133, 258.707], [745.31, 273.516], [757.382, 285.661], [771.0, 295.104], [786.528, 298.963], [803.287, 298.28], [820.299, 292.651], [835.5, 284.333], [847.558, 272.277], [855.689, 256.945], [861.119, 239.878], [865.258, 221.967], [868.033, 203.576], [737.08, 170.426], [744.747, 164.545], [754.731, 163.208], [765.006, 165.528], [774.572, 170.176], [800.37, 171.892], [812.763, 168.971], [825.469, 168.597], [837.53, 172.651], [846.962, 179.961], [786.077, 188.028], [784.649, 198.089], [783.057, 208.051], [781.484, 218.288], [771.829, 230.039], [776.69, 231.791], [781.95, 233.136], [787.9, 232.782], [794.077, 232.314], [747.713, 189.047], [754.601, 184.644], [763.581, 185.357], [771.085, 191.701], [762.755, 193.421], [753.766, 192.909], [805.804, 194.866], [814.228, 189.444], [823.818, 190.612], [831.668, 195.959], [823.818, 198.623], [814.256, 197.783], [762.705, 256.094], [769.212, 248.415], [776.974, 244

In [21]:
# # 计算左上、右下坐标
# sample1_keypoints_x = [i[0] for i in sample1_keypoints]
# sample1_keypoints_y = [i[1] for i in sample1_keypoints]
# x_left = min(sample1_keypoints_x)
# x_right = max(sample1_keypoints_x)
# y_low = min(sample1_keypoints_y)
# y_high = max(sample1_keypoints_y)

# print(x_left,y_high, x_right, y_low)


In [22]:
pose_local_visualizer = PoseLocalVisualizer(radius=1, link_color = 'red')


image = Image.open(sample1['img_path'])  # 打开图像文件
image = np.array(image) # 将图像转换为 NumPy 数组


# PoseDataSample存的是所有目标关键点信息
gt_pose_data_sample = PoseDataSample() 

# # 将构造的真实关键点存入PoseDataSample中
gt_instances = InstanceData() # InstanceData对象中存的是一个目标关键点信息
gt_instances.keypoints = np.array([sample1_keypoints])
gt_instances.bboxes = np.array([sample1_bbox])

# gt_instances.bboxes = np.array([[30,30, 300, 300]]) 
gt_pose_data_sample.gt_instances = gt_instances 

# 构造一个目标的预测关键点标签存入InstanceData
pred_instances = InstanceData()
pred_instances.keypoints = np.array([[[150, 150], [250, 250], [450, 450], 
                                      [85, 85]]])
pred_instances.score = np.array([[1,1,1,1]]) # 构造每个关键点标签权重
# 将该目标预测关键点存入PoseDataSample中
gt_pose_data_sample.pred_instances = pred_instances 

# 传入图片、标签、预测、配置，开始画图
pose_local_visualizer.add_datasample('image', image,
                        gt_pose_data_sample,
                        out_file='out_file.jpg',
                        show=True,
                        draw_bbox = True )

array([[[116, 103, 123],
        [116, 103, 121],
        [116, 103, 121],
        ...,
        [  1,   1,   9],
        [  1,   1,   9],
        [  1,   1,   9]],

       [[116, 103, 123],
        [116, 103, 123],
        [116, 103, 123],
        ...,
        [  1,   1,   9],
        [  1,   1,   9],
        [  1,   1,   9]],

       [[116, 103, 123],
        [116, 103, 123],
        [116, 103, 123],
        ...,
        [  1,   1,   9],
        [  1,   1,   9],
        [  1,   1,   9]],

       ...,

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]]